In [1]:
using LowLevelFEM, LinearAlgebra
gmsh.initialize()

gmsh.open("tarto.geo")

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (4), wrong dep version loaded (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
Error   : Gmsh has not been initialized


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00164752s, CPU 0.001648s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00135075s

┌ Warning: Number of threads(1) ≠ logical threads in CPU(4).
└ @ LowLevelFEM ~/Dokumentumok/GitHub/LowLevelFEM.jl/src/general.jl:104


Info    : Reading 'tarto.geo'...
Info    : Meshing 1D...- Splitting solids                                                                                             
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (TrimmedCurve)
Info    : [ 10%] Meshing curve 6 (TrimmedCurve)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (TrimmedCurve)
Info    : [ 10%] Meshing curve 10 (TrimmedCurve)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Circle)
Info    : [ 20%] Meshing curve 13 (Circle)
Info    : [ 20%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Circle)
Info    : [ 20%] M

In [2]:
mat = material("part")
prob = Problem([mat])

Problem("tarto", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("part", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 41176)

In [ ]:
supp1 = displacementConstraint("base", uz=0)
supp2 = displacementConstraint("hole1", ux=0, uy=0)
supp3 = displacementConstraint("perim", uz=-0.00001)

load1 = load("hole2", fy=-5)

In [ ]:
q = solveDisplacement(prob, [load1], [supp1, supp2, supp3]);

In [ ]:
f = loadVector(prob, [load1]);

In [ ]:
showDoFResults(f, name="load")

In [ ]:
showDoFResults(q, name="displacement")

In [ ]:
S = solveStress(q);

In [ ]:
showElementResults(S, name="stress")

In [ ]:
S = elementsToNodes(S);

In [ ]:
S1 = nodesToElements(S, onPhysicalGroup="hole2");

In [ ]:
ex = VectorField(prob, "hole2", [1, 0, 0])
ey = VectorField(prob, "hole2", [0, 1, 0])
ez = VectorField(prob, "hole2", [0, 0, 1]);

In [ ]:
n1 = normalVector(prob, "hole2");

In [ ]:
showElementResults(n1, name="normal")

In [ ]:
p1 = S1 * n1
p1x = p1 * ex
p1y = p1 * ey
p1z = p1 * ez;

In [ ]:
showElementResults(p1, name="traction")

In [ ]:
f1 = loadVector(prob, [load("hole2", fx=p1x, fy=p1y, fz=p1z)]);

In [ ]:
resultant(f, "hole2")

In [ ]:
fr1 = resultant(f1, "hole2")

In [ ]:
norm(fr1)

In [ ]:
integrate(prob, "hole2", p1x)

In [ ]:
integrate(prob, "hole2", p1y)

In [ ]:
integrate(prob, "hole2", p1z)

In [ ]:
showDoFResults(f1, name="load2")

In [ ]:
openPostProcessor()

In [ ]:
gmsh.finalize()